<a href="https://colab.research.google.com/github/Omare04/StockPulse/blob/main/StockPulsePreprocesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initialize Git

In [ ]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [ ]:
!git remote add origin https://github.com/Omare04/StockPulse.git

In [ ]:
!git config --global user.email "omarelmasaoudi04@gmail.com"
!git config --global user.name "Omare04"


In [31]:
from google.colab import userdata
github_key = userdata.get('PAT_token')
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!git push -u origin main


Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 759 bytes | 759.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/Omare04/StockPulse.git
   1be2596..fe1639b  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [ ]:
!git remote set-url origin https://{github_key}@github.com/Omare04/StockPulse.git


In [45]:
!echo "*.gsheet" >> .gitignore
!echo "*.gdoc" >> .gitignore
!echo "drive/MyDrive/" >> .gitignore
!echo "sample_data/" >> .gitignore
!/content/drive/MyDrive/Colab\ Notebooks/StockPulse.ipynb




/content/drive/MyDrive/Colab Notebooks/StockPulse.ipynb: line 1: nbformat:4: command not found


In [ ]:
!git add .gitignore


In [ ]:
!git add .
!git commit -m "Initial commit"
!git branch -M main
!git push -u origin main


[main 1be2596] Initial commit
 1 file changed, 4 insertions(+)
Enumerating objects: 25, done.
Counting objects: 100% (25/25), done.
Delta compression using up to 2 threads
Compressing objects: 100% (18/18), done.
Writing objects: 100% (25/25), 7.06 KiB | 1.41 MiB/s, done.
Total 25 (delta 6), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (6/6), done.
To https://github.com/Omare04/StockPulse.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


### Install Dependencies and import packages



In [ ]:

!pip install pandas numpy psycopg2 pinecone-client sentence-transformers nltk transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import json
import pandas as pd
import numpy as np
import re
import psycopg2
from datetime import datetime as dt
from pinecone import Pinecone
from google.colab import drive
from sentence_transformers import SentenceTransformer
import nltk
from transformers import pipeline
nltk.download('punkt_tab')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Load Embeddings Model

In [ ]:
model = SentenceTransformer('ProsusAI/finbert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def embedd_model(text):
  embedding = model.encode(text)
  return embedding

### Connect to pinecone vector store


In [ ]:
api_key = userdata.get('pinecone_vector_store')
pc = Pinecone(api_key=api_key)
index = pc.Index("stockpulse-articles")

In [ ]:
def insert_embeddings_to_vector(embeddings, articles):
    vectors = []
    for i, (embedding, article) in enumerate(zip(embeddings, articles)):
        vectors.append({
            "id": f"article_{i}",
            "values": embedding,
            "metadata": {"text": article}
        })

    # Batch upsert to Pinecone
    index.upsert(vectors=vectors, namespace="articles")
    print(f"Inserted {len(vectors)} articles into Pinecone.")


### Load Articles And stocks


In [ ]:
articles = pd.read_json('./drive/MyDrive/StockPulse/articles/stock_articles_DEC31.json')
stocks = pd.read_json("./drive/MyDrive/StockPulse/stocks/marketmood_stocks.json")

####Initialize pandas dataframes for each object

In [ ]:
article_df = pd.DataFrame(articles)
stock_df = pd.DataFrame(stocks)

In [ ]:
print(article_df["article"])

0       {'title': '35% of Warren Buffett's $296 Billio...
1       {'title': 'Nvidia Stock Investors Just Got Bad...
2       {'title': 'Meet the Magnificent Vanguard ETF W...
3       {'title': 'How smartphones powered the AI boom...
4       {'title': '35% of Warren Buffett's $296 Billio...
                              ...                        
2412    {'title': 'Magnificent Seven" Dominate 2024: N...
2413    {'title': 'The Top ETF of December & its Best-...
2414    {'title': 'Mag 7 is turning to 'lag 7': How to...
2415    {'title': 'Is Tesla's Stock Flying Too High? H...
2416    {'title': 'Why Tesla (TSLA) Shares Are Falling...
Name: article, Length: 2417, dtype: object


### Clean Data by removing Emails, links, and addresses

In [ ]:

def clean_text(text):
    # Remove emails
    text = re.sub(r'\b\S+@\S+\.\S+\b', '', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove full addresses
    text = re.sub(r'\d{1,5} [\w\s]+, [A-Za-z\s]+, [A-Z]{2} \d{5}', '', text)

    # Remove phone numbers
    text = re.sub(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{0,4}', '', text)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    return text



In [ ]:
article_df["article"] = article_df["article"].apply(lambda x: {**x, "content": clean_text(x["content"])})

#### Check for consistency

In [ ]:
import textwrap

content = article_df["article"][2400]["content"]

# Use textwrap to format the text
formatted_content = textwrap.fill(content, width=80)

print(formatted_content)


When the curtain closes on 2024 in just four days, it'll likely go down as
another phenomenal year for Wall Street and investors. On a year-to-date basis,
as of the closing bell on Dec. 23, the ageless Dow Jones Industrial Average,
benchmark S&P 500, and growth-powered Nasdaq Composite have respectively
increased by 14%, 25%, and 32%. No shortage of catalysts is responsible for
propelling Wall Street's major stock indexes to new heights, including the rise
of artificial intelligence (AI), better-than-expected corporate earnings, and
Donald Trump's November victory. Don't forget, the Dow, S&P 500, and Nasdaq
Composite all rocketed higher during Trump's first term in the Oval Office. But
perhaps Wall Street's unsung hero, which really helped to fuel the 2024 bull
market rally, is the euphoria regarding stock splits. A stock split is a
mechanism that allows publicly traded companies to adjust their share price and
outstanding share count by the same factor. Keep in mind that changing a
co

###Split Articles by sentence

In [ ]:
from nltk.tokenize import sent_tokenize

def split_by_sentences(text, id):
    article_sentences = {}
    sentences = sent_tokenize(text)
    article_sentences = {"id": id, "sentences": sentences}
    return article_sentences



In [ ]:
articles_entities = []
for i in range(len(article_df)):
    text = article_df["article"][i]["content"]
    article_id = article_df["_id"][i]["$oid"]
    sentences = split_by_sentences(text=text, id=article_id)
    articles_entities.append(sentences)

In [ ]:
print(articles_entities[0:2])

[{'id': '676c877c746aa6fe346a31cc', 'sentences': ['Since Warren Buffett became the CEO of Berkshire Hathaway (NYSE: BRK.A)(NYSE: BRK.B) in 1965, its stock has delivered a compound annual return of 19.8%.', 'That would have been enough to turn a $1,000 investment into $42.5 million, whereas the same investment in the S&P 500 would have grown to just $325,053 over the same period.', "Buffett's long-term investing strategy is simple.", 'He buys into companies with steady growth, robust profitability, strong management teams, and shareholder-friendly initiatives like stock buyback programs and dividend schemes, which help to compound his returns over time.', "You won't ever see Buffett chasing the latest stock market trend -- not even one as powerful as artificial intelligence (AI).", "With that said, three of the existing stocks in Berkshire's $296 billion portfolio of publicly traded securities are integrating AI into their legacy businesses in very unique ways.", "Amazon (NASDAQ: AMZN) 

### NER for entities in article sentences

In [ ]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
def extract_entities(sentence):
  entities = ner_pipeline(sentence)
  return entities

####Convert article_entities to Dataset

In [ ]:
from datasets import Dataset


### Must convert articles_entities to pandas dataframe for more efficient computation

# Check the dataset
print(dataset)


Dataset({
    features: ['id', 'sentences'],
    num_rows: 2417
})


####Apply NER to each sentence for each article

In [ ]:
print(articles_entities[0])

# for obj in articles_entities:
#     sentences = obj["sentences"]
#     extracted_entities = []

#     for i, sentence in enumerate(sentences):
#         entities = extract_entities(sentence)
#         extracted_entities.append({"sentence": i, "entities": entities})

#     # Add to the current article object
#     obj["extracted_entities"] = extracted_entities


{'id': '676c877c746aa6fe346a31cc', 'sentences': ['Since Warren Buffett became the CEO of Berkshire Hathaway (NYSE: BRK.A)(NYSE: BRK.B) in 1965, its stock has delivered a compound annual return of 19.8%.', 'That would have been enough to turn a $1,000 investment into $42.5 million, whereas the same investment in the S&P 500 would have grown to just $325,053 over the same period.', "Buffett's long-term investing strategy is simple.", 'He buys into companies with steady growth, robust profitability, strong management teams, and shareholder-friendly initiatives like stock buyback programs and dividend schemes, which help to compound his returns over time.', "You won't ever see Buffett chasing the latest stock market trend -- not even one as powerful as artificial intelligence (AI).", "With that said, three of the existing stocks in Berkshire's $296 billion portfolio of publicly traded securities are integrating AI into their legacy businesses in very unique ways.", "Amazon (NASDAQ: AMZN) i

### Stock Ticker Frequency Counter

In [ ]:

def company_freq_counter(text):
  return True
